### Algorithm As A Sequence
* Takes in the user creds, which comes in as an user object, it then segregates the user into regular or premium.
* For Regular Users :
    * It Takes the rating of restaurants and displays them in the descending order of rating
    * It gets this rating by performing a sentiment analysis of the reviews given by the users
* For Premium Users : 
    * It takes in the BMI of the users, calculates the ideal weight and the weight gap and from that calculates the per day calories that should be taken to reach desired weight over a span of D days
    * Now it splits the calories proportionately for carbohydrates proteins and fats and suggests user interested foods accordingly

In [170]:
from math import ceil
from tensorflow import keras
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
import numpy as np
import copy

class BMIFilter:
    def __init__(self, height, weight, calories_intake, strict_diet=True):
        self.height = height
        self.weight = weight
        self.calories_intake=calories_intake
        self.strict_diet=strict_diet
        self.Ddays = 0
        self.share = None
        
    def __getIdealWeight(self):
        return 22*self.height**2
    
    def __getRevisedCaloriesPerDay(self, weight_gap, per_day_cal=2500, strict=True):  #Per day calories intake calculated from the user data during data gathering period
        max_revised_cals = per_day_cal - 500       #A minimum of 500 calories per day has to be burnt
        if strict:
            diet_days = 1
            rc = per_day_cal - weight_gap*7700/diet_days
            while rc<500:                         #Minimum of 500 calories per day has to be consumed
                diet_days += 1
                rc = per_day_cal - weight_gap*7700/diet_days 
            return diet_days, rc
        else:
            return 7700*weight_gap/500, max_revised_cals 


    def __getCaloriesSplit(self, total_calories):
        carbs_split = 0.45
        proteins_split = 0.35
        fats_split = 0.2
        return {'carbs' : carbs_split*total_calories, 'proteins':proteins_split*total_calories, 'fats':fats_split*total_calories}
    def __call__(self, food_items, topn=3):
        wi = self.__getIdealWeight()
        D, rc = self.__getRevisedCaloriesPerDay(self.weight-wi,self.calories_intake,self.strict_diet)
        self.Ddays = ceil(D)
        share = self.__getCaloriesSplit(rc)
        self.share = share
        filtered_list = {'carbs':[], 'proteins':[], 'fats':[]}
        for mn in filtered_list.keys():
            food_cons = [ fi for fi in sorted(food_items, key=lambda x: eval(f'x.{mn}'), reverse=True) if eval(f'fi.{mn}')<= share['carbs']][:topn]
            filtered_list[mn] = food_cons
            for i in food_cons:
                food_items.remove(i)

        return filtered_list
            
            
PRESENT_LOCATION, PRESENT_TIME, PRESENT_SEASON = 123, 2, 1   #Locations of size 300, time of size :4, morning, afternoon, evening, night, season of size :3, summer, rainy, winter    
class PreferenceFilter:
    def __init__(self, preferred_foods):
        self.pref_food = preferred_foods
        self.pref_model = None
    def __make_model(self):
        Xinp = Input((8,))  #8 Features : Key Ingredients (1-5), Location(1), Time(1), Season(1) 
        X = Dense(10, activation='relu')(Xinp)
        X = Dense(5, activation='sigmoid')(X)
        X = Dense(1, activation='sigmoid')(X) #One Binary Output mentioning whether user likes the dish or not
        
        model = Model(inputs=Xinp, outputs=X)
        model.compile('adam', 'binary_crossentropy', metrics=['accuracy'])
        self.pref_model = model
        return model
    def __preprocess(self, xinp=None):
        if type(xinp) == type(None):
            x = []
            y = []
            for fi in self.pref_food:
                x.append([fi.keing1, fi.keing2, fi.keing3, fi.keing4, fi.keing5, PRESENT_LOCATION, PRESENT_TIME, PRESENT_SEASON])
                y.append(1)
            #negative samples
            xn = []
            yn = []
            for i in range(3):
                fi = Food()
                xn.append([fi.keing1, fi.keing2, fi.keing3, fi.keing4, fi.keing5, PRESENT_LOCATION, PRESENT_TIME, PRESENT_SEASON])
                yn.append(0)
            x += xn
            y += yn
            return np.array(x), np.array(y) 
        else:
            x = []
            for fi in xinp:
                x.append([fi.keing1, fi.keing2, fi.keing3, fi.keing4, fi.keing5, PRESENT_LOCATION, PRESENT_TIME, PRESENT_SEASON])
            return np.array(x)
            
            
    def __train(self, x, y, eps=100):
        model = self.pref_model
        model.fit(x, y, epochs=eps, verbose=0)
        self.pref_model = model
        return model
    def __call__(self, xinp):
        x, y = self.__preprocess()
        self.__make_model()
        self.__train(x, y, eps=100)
        filtered_list2 = {'carbs':[], 'proteins':[], 'fats':[]}
        for fi in filtered_list2.keys():
            xtest = self.__preprocess(xinp[fi])
            if xtest.shape[0] != 0:
                filtered_foods = [xinp[fi][i] for i, v in enumerate(self.pref_model(xtest)) if v >=0.7]  #considering only interested foods
            else:
                filtered_foods = []
            filtered_list2[fi] += filtered_foods
        return filtered_list2
class Food:
    def __init__(self, food_id = 'FI101', name=None, carbs=None, proteins=None, fats=None):
        self.food_id =  food_id
        self.name = name
        self.carbs = carbs
        self.proteins = proteins
        self.fats = fats
        self.keing1 = np.random.randint(1, 11) #KeyIngredients Size : 10
        self.keing2 = np.random.randint(1, 11)
        self.keing3 = np.random.randint(1, 11)
        self.keing4 = np.random.randint(1, 11)
        self.keing5 = np.random.randint(1, 11)
    def __set_attributes(self):
        ##Acceses data base and for every food item attributes are extracted and set
        return

class util:
    def __init__(self):
        pass
    def len_(self, obj):
        c = 0
        for v in obj.values():
            c += len(v)
        return c
    def disp_(self, obj):
        if type(obj) == type([]):
            print("Foods:")
            for c in obj:
                print("\t",c.name)
        else:
            for c in obj.keys():
                print(c,":")
                for v in obj[c]:
                    print("\t",v.name)
    
        

In [176]:
preferred_foods = [Food() for i in range(10)]
food_items = [Food(name='dosa', carbs=230, proteins=120, fats=50), Food(name='idli', carbs=270, proteins=80, fats=10), Food(name='Puri', carbs=70, proteins=60, fats=150), Food(name='vada', carbs=50, proteins=20, fats=20), Food(name='Pizza', carbs=170, proteins=20, fats=250)]

In [177]:
print(f"Total List of Food Items unfiltered : {len(food_items)}")
special_calc = util()
special_calc.disp_(food_items)
bmi_filt = BMIFilter(1.6, 70, 3000, strict_diet=False)
pref_filt = PreferenceFilter(preferred_foods)


filtered_list1 = bmi_filt(copy.deepcopy(food_items))
print(f"Total List of Food Items BMI filtered : {special_calc.len_(filtered_list1)}")
special_calc.disp_(filtered_list1)
filtered_list2 = pref_filt(filtered_list1)
print(f"Total List of Food Items Preference filtered : {special_calc.len_(filtered_list2)}")
special_calc.disp_(filtered_list2)

Total List of Food Items unfiltered : 5
Foods:
	 dosa
	 idli
	 Puri
	 vada
	 Pizza
Total List of Food Items BMI filtered : 5
carbs :
	 idli
	 dosa
	 Pizza
proteins :
	 Puri
	 vada
fats :
Total List of Food Items Preference filtered : 0
carbs :
proteins :
fats :


values:
	 dosa
	 idli
	 Puri
	 vada
	 Pizza
carbs :
	 idli
	 dosa
	 Pizza
proteins :
	 Puri
	 vada
fats :
carbs :
	 idli
	 dosa
	 Pizza
proteins :
	 Puri
fats :


(None, None, None)